# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
df = pd.read_csv("../output_data/cities.csv")
df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Atuona,-9.8000,-139.0333,76.53,78,30,17.25,PF,1665826901
1,Leningradskiy,69.3833,178.4167,11.12,86,71,5.26,RU,1665826902
2,Busselton,-33.6500,115.3333,57.85,65,20,13.06,AU,1665826902
3,Vaini,-21.2000,-175.2000,69.96,100,20,2.30,TO,1665826902
4,Albany,42.6001,-73.9662,41.54,90,4,0.92,US,1665826902


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
df.columns

Index(['City', 'Latitude', 'Longitude', 'Max Temperature', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations = df[["Latitude", "Longitude"]]
weight = df["Humidity"]

# Add a Heatmap layer to map
fig = gmaps.figure(center = (0, 0), zoom_level = 1)
heatmap_layer = gmaps.heatmap_layer(locations, weights = weight)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions:
1. A max temperature lower than 80 degrees but higher than 70
2. Wind speed less than 10 mph
3. Zero cloudiness
4. Humidity is less than 30
* Drop any rows with null values.

In [11]:
df.columns

Index(['City', 'Latitude', 'Longitude', 'Max Temperature', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             215 non-null    object 
 1   Latitude         215 non-null    float64
 2   Longitude        215 non-null    float64
 3   Max Temperature  215 non-null    float64
 4   Humidity         215 non-null    int64  
 5   Cloudiness       215 non-null    int64  
 6   Wind Speed       215 non-null    float64
 7   Country          213 non-null    object 
 8   Date             215 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 15.2+ KB


In [18]:
df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Atuona,-9.8000,-139.0333,76.53,78,30,17.25,PF,1665826901
1,Leningradskiy,69.3833,178.4167,11.12,86,71,5.26,RU,1665826902
2,Busselton,-33.6500,115.3333,57.85,65,20,13.06,AU,1665826902
3,Vaini,-21.2000,-175.2000,69.96,100,20,2.30,TO,1665826902
4,Albany,42.6001,-73.9662,41.54,90,4,0.92,US,1665826902


In [73]:
# Narrow down ideal weather conditions
ideal_temp = (df["Max Temperature"] < 80) & (df["Max Temperature"] > 70)
ideal_wind = (df["Wind Speed"] < 10)
ideal_cloud = (df["Cloudiness"] == 0)
ideal_humidity = (df["Humidity"] < 30)
ideal_weather = df.loc[ideal_temp & ideal_wind & ideal_cloud & ideal_humidity]

ideal_weather.info()
print("These ideal weather conditions were too narrow for the existing data set")
ideal_weather

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             0 non-null      object 
 1   Latitude         0 non-null      float64
 2   Longitude        0 non-null      float64
 3   Max Temperature  0 non-null      float64
 4   Humidity         0 non-null      int64  
 5   Cloudiness       0 non-null      int64  
 6   Wind Speed       0 non-null      float64
 7   Country          0 non-null      object 
 8   Date             0 non-null      int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 0.0+ bytes
These ideal weather conditions were too narrow for the existing data set


,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date


In [74]:
# Narrow down ideal weather conditions
ideal_temp = (df["Max Temperature"] <= 90) & (df["Max Temperature"] >= 60)
ideal_wind = (df["Wind Speed"] <= 30)
ideal_cloud = (df["Cloudiness"] <= 30)
ideal_humidity = (df["Humidity"] <= 30)
ideal_weather = df.loc[ideal_temp & ideal_wind & ideal_cloud & ideal_humidity]

# ideal_df = df.loc[(df["Wind Speed"] <= 10)][(df["Cloudiness"] == 0)][(df["Max Temperature"] < 80)][(df["Max Temperature"] > 70)][(df["Humidity"] < 30)].dropna()

# Drop null values
ideal_weather = ideal_weather.dropna()

ideal_weather.info()
ideal_weather

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 26 to 98
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             3 non-null      object 
 1   Latitude         3 non-null      float64
 2   Longitude        3 non-null      float64
 3   Max Temperature  3 non-null      float64
 4   Humidity         3 non-null      int64  
 5   Cloudiness       3 non-null      int64  
 6   Wind Speed       3 non-null      float64
 7   Country          3 non-null      object 
 8   Date             3 non-null      int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 240.0+ bytes


,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
26,Zomba,-15.3860,35.3188,84.09,19,0,7.92,MW,1665826906
89,Phalaborwa,-23.9430,31.1411,79.88,27,4,12.35,ZA,1665826917
98,Bo`ka,40.8108,69.1986,82.69,9,13,3.85,UZ,1665826918


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [75]:
# store into variable name hotel_df
hotel_df = ideal_weather

# add "Hotel Name" column
hotel_df["Hotel Name"] = ""

hotel_df.info()
hotel_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 26 to 98
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             3 non-null      object 
 1   Latitude         3 non-null      float64
 2   Longitude        3 non-null      float64
 3   Max Temperature  3 non-null      float64
 4   Humidity         3 non-null      int64  
 5   Cloudiness       3 non-null      int64  
 6   Wind Speed       3 non-null      float64
 7   Country          3 non-null      object 
 8   Date             3 non-null      int64  
 9   Hotel Name       3 non-null      object 
dtypes: float64(4), int64(3), object(3)
memory usage: 264.0+ bytes


,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
26,Zomba,-15.3860,35.3188,84.09,19,0,7.92,MW,1665826906,
89,Phalaborwa,-23.9430,31.1411,79.88,27,4,12.35,ZA,1665826917,
98,Bo`ka,40.8108,69.1986,82.69,9,13,3.85,UZ,1665826918,


In [89]:
# Set parameters to search for hotels with 5000 meters
Lat = hotel_df["Latitude"]
Lng = hotel_df["Longitude"]
coords = (f"({Lat},{Lng})")
city = hotel_df["City"]


# Hit the Google Places API for each city's coordinates.
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row["City"]

    parameters = {
    "key": g_key,
    "radius": 5000,
    "location": coords,
    "type": "lodging"
   }

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response_json = requests.get(url, params = parameters).json()

    # hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]

    # Store the first Hotel result into the DataFrame.
    try:
        print(f"The first closest hotel to : {city}")
        hotel_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing information... skipping.")
    print("---")

Missing information... skipping.
---
Missing information... skipping.
---
Missing information... skipping.
---


In [69]:
# Plot markers on top of the heatmap

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
